## Estimating the impact of AuxTel mount oscillations on Image Quality.
Craig Lage 04-May-22

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION

In [ ]:
## SOMETHING STILL NOT RIGHT HERE ##
"""
def dAzdT(ALT, AZ):
    LAT = AUXTEL_LOCATION.lat
    dHdT = 360.0 * 3600.0 / (86400.0 - 236.0)
    sinDec = np.sin(LAT) * np.sin(ALT) + np.cos(LAT) * np.cos(ALT) * np.cos(AZ)
    cosDec = np.sqrt(1.0 - sinDec**2)
    tanDec = sinDec / cosDec
    sinHA = - np.sin(AZ) * np.cos(ALT) / cosDec
    cosHA = np.sqrt(1.0 - sinHA**2)
    #cosHA = (np.sin(ALT) - np.sin(LAT) * sinDec) / (cosDec * np.cos(LAT))
    #sinHA = np.sqrt(1.0 - cosHA**2)
    daltdt = cosDec * sinHA * dHdT
    print(np.arcsin(sinDec)*180.0/np.pi, np.arcsin(sinHA)*180.0/np.pi)
    dazdt = np.tan(LAT) / (np.sin(AZ) * np.cos(ALT)**2) * (sinDec + 1.0) * daltdt
    return abs(dazdt.value)
"""
def dAzdT(ALT, AZ):
    deltaT = 10.0
    time = datetime.now()
    shiftedTime = time + timedelta(seconds=deltaT)
    altAz = AltAz(obstime=time, location=AUXTEL_LOCATION)
    shiftedAltAz = AltAz(obstime=shiftedTime, location=AUXTEL_LOCATION)
    skyLocation = SkyCoord(AZ, ALT, frame=altAz)
    az1 = skyLocation.altaz.az.deg
    shiftedSkyLocation = skyLocation.transform_to(shiftedAltAz)
    az2 = shiftedSkyLocation.altaz.az.deg
    deltaAz = abs(az1 - az2)
    if deltaAz > 1.0:
        deltaAz = abs(deltaAz - 360.0)
    azSpeed = abs(deltaAz) / deltaT * 3600.0
    if azSpeed > 500.0:
        return 0.0
    else:
        return azSpeed


In [ ]:
SkyCoord?

In [ ]:
deltaT = 10.0
time = datetime.now()
shiftedTime = time + timedelta(seconds=deltaT)
ALT = Angle(50.0 * u.deg)
AZ = Angle(0.01 * u.deg)
altAz = AltAz(obstime=time, location=AUXTEL_LOCATION)
shiftedAltAz = AltAz(obstime=shiftedTime, location=AUXTEL_LOCATION)
skyLocation = SkyCoord(AZ, ALT, frame=altAz)
az1 = skyLocation.altaz.az.deg
if az1 > 180.0:
    az1 -= 360.0
shiftedSkyLocation = skyLocation.transform_to(shiftedAltAz)
az2 = shiftedSkyLocation.altaz.az.deg
if az2 > 180.0:
    az2 -= 360.0
print(az1, az2)
deltaAz = az1 - az2
azSpeed = abs(deltaAz) / deltaT * 3600.0
period = 675.0 / azSpeed
print(azSpeed, period)

In [ ]:
skyLocation.icrs.dec.deg

In [ ]:
shiftedSkyLocation.icrs.dec.deg

In [ ]:
az1 = skyLocation.altaz.az

In [ ]:
az2 = shiftedSkyLocation.altaz.az

In [ ]:
sep = skyLocation.separation(shiftedSkyLocation)

In [ ]:
print(sep)

In [ ]:
shiftedSkyLocation.altaz.az - skyLocation.altaz.az

In [ ]:
time

In [ ]:
shiftedTime

In [ ]:
def dAzdT(ALT, AZ):
    skyLocation = SkyCoord(origin.getRa().asRadians(), origin.getDec().asRadians(), unit=u.rad)
altAz = AltAz(obstime=vi.date.toPython(), location=AUXTEL_LOCATION)
obsAltAz = skyLocation.transform_to(altAz)
shiftInArcsec = 50.0
altShifted = SkyCoord(obsAltAz.az, obsAltAz.alt + Angle(shiftInArcsec * u.arcsec), frame=altAz)
altShiftedSpherePoint = SpherePoint(altShifted.icrs.ra.deg*geom.degrees, altShifted.icrs.dec.deg*geom.degrees)
azShifted = SkyCoord(obsAltAz.az + Angle(shiftInArcsec / np.cos(obsAltAz.alt) * u.arcsec), obsAltAz.alt, frame=altAz)
azShiftedSpherePoint = SpherePoint(azShifted.icrs.ra.deg*geom.degrees, azShifted.icrs.dec.deg*geom.degrees)
originPixel = wcs.skyToPixel(origin)


In [ ]:
dAzdT(Angle(65.04 * u.deg), Angle(190.1 * u.deg))

In [ ]:
AUXTEL_LOCATION.lat.deg

In [ ]:
nx = 50
ny = 100

alts = np.linspace(5.0, 80.0, nx)
azs = np.linspace(-179.9, 179.9, ny)
alt_1, az_1 = np.meshgrid(alts, azs)
z = np.zeros([ny, nx])
for i, alt in enumerate(alts):
    for j, az in enumerate(azs):
        z[j,i] = dAzdT(Angle(alt * u.deg), Angle(az * u.deg))
        #print(alt, az, z[i,j])
        

In [ ]:
z.max()

In [ ]:
plt.contourf(alt_1, az_1, z)
plt.colorbar()

In [ ]:
# Plot as a polar plot
r, theta = np.meshgrid(alts, azs)

fig, ax = plt.subplots(subplot_kw=dict(projection='polar'))
ax.set_title("Azimuth axis speed (arcseconds/second)")
ax.contourf(theta, r, z)
ax.set_rorigin(90.0)

#pc = ax.pcolormesh(theta, r, z)

#fig.colorbar(pc)
#ax.set_xticks([0.0,180.0], color='white')
ax.tick_params(axis='x', labeltop=True, labelbottom=False, colors='yellow')
ax.tick_params(axis='y', colors='yellow')


In [ ]:
def AverageDeviation(period, amplitude, exptime):
    nTrials = 100
    times = np.linspace(0, exptime, 100)
    deviations = []
    for i in range(nTrials):
        phase = 2.0 * np.pi * np.random.random()
        motion = np.cos(2.0 * np.pi * times / period + phase)
        deviation = motion.max() - motion.min()
        deviations.append(deviation)
        #print(i, phase, deviation)
    return np.mean(deviations)
        
def ImageImpact(ALT, AZ): 
    exptime = 30.0 # seconds
    coggingPeriod = 675.0 # arcseconds
    azSpeed = dAzdT(ALT, AZ)
    period = coggingPeriod / azSpeed
    azFreq = azSpeed / coggingPeriod
    amplitude = 12.96 * azFreq # Empirical relation - arcseconds
    #print(azSpeed, period, azFreq, amplitude)
    imageImpact = AverageDeviation(period, amplitude, exptime) * np.cos(ALT)
    return abs(imageImpact)

In [ ]:
AverageDeviation(10.0, 1.0, 30.0)

In [ ]:
ImageImpact(Angle(30.0 * u.deg), Angle(0.01 * u.deg))

In [ ]:
alts = np.linspace(5.0, 80.0, 50)
azs = np.linspace(-179.9, 179.9, 100)
alt_1, az_1 = np.meshgrid(alts, azs)
z = np.zeros([100, 50])
for i, alt in enumerate(alts):
    for j, az in enumerate(azs):
        z[j,i] = ImageImpact(Angle(alt * u.deg), Angle(az * u.deg))

plt.contourf(alt_1, az_1, z)
plt.colorbar()  
"""
# Plot as a polar plot
r, theta = np.meshgrid(alts, azimuths)

fig, ax = plt.subplots(subplot_kw=dict(projection='polar'))
ax.set_title("Image Impact (arcseconds)")
ax.contourf(theta, r, z)
ax.set_rorigin(90.0)

pc = ax.pcolormesh(theta, r, z)

fig.colorbar(pc)
#ax.set_xticks([0.0,180.0], color='white')
ax.tick_params(axis='x', labeltop=True, labelbottom=False, colors='red')
ax.tick_params(axis='y', colors='red')
"""